In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX


In [ ]:
%cd /content/drive/MyDrive/
!mkdir -p EndoMapNet
%cd EndoMapNet/
!git init
!git clone https://github.com/YJonmo/EndoMapNet
%cd EndoMapNet

In [ ]:
######################## You could skip this as it uploads the pretrained model####################
#This is the pretrained model on 8000 images of a 3D printed knee. The resolution of the image is 256*256.
#The 8000 images may not be enough for achieving desirable results. More training maybe needed. 
#It can be used for training your model or for demonstration of the output. 
!gdown https://drive.google.com/u/1/uc?id=1J1MFm9tudt-YJWa6b32Whjb10na4sUgT&export=download
!mkdir -p PreTrained
!unzip PreTrained.zip -d PreTrained

In [ ]:
%%bash
##################### This is a demo for training the network on the images ##################################
DATA_Folder=/content/drive/MyDrive/EndoMapNet/EndoMapNet/DataSets/
#now=$(date +"%T")
OUTPUT=TrainingFolder
mkdir -p $OUTPUT
Splits=3DPrintedKnee
weight=PreTrained

python train.py --data_path $DATA_Folder --split $Splits  \
	--log_dir $OUTPUT --png --no_eval --dataset Custom \
	--height 256 --width 256 --batch_size 8 --frame_ids 0 2 \
	--pose_model_input all  --num_epochs 10 --use_stereo \
  --load_weights_folder $weight \
  --models_to_load pose_encoder encoder depth pose \
	--use_pose 1 

In [ ]:
%%bash 
##################### This is a demo for predicting depth from images ##################################
Weight=./PreTrained  # or ./TrainingFolder/mdp/models/weights_XX 
DATA_Folder=./DataSets/TestDepth
python test_simple.py --image_path $DATA_Folder --model_name $Weight 

In [ ]:
import PIL.Image as pil
from matplotlib import pyplot as plt
image=pil.open('./DataSets/TestDepth/02050.png')
depth=pil.open('./DataSets/TestDepth/02050_depth.jpeg')
plt.imshow(image)
plt.title('Input image')
plt.figure()
plt.title('Guessed depth')
plt.imshow(depth)

In [ ]:
%%bash
##################### This is a demo for predicting poses of a sequence if images ##################################
Splits='2020-09-18-14-03-36'
DATA_Folder=/content/drive/MyDrive/EndoMapNet/EndoMapNet/DataSets/
Weight=PreTrained
python evaluate.py --split $Splits  --load_weights_folder $Weight \
  --png --dataset Custom --data_path $DATA_Folder \
  --height 256 --width 256 --batch_size 1 --use_stereo --frame_ids  0 2 \
  --pose_model_type separate_resnet --pose_model_input all \
  --models_to_load pose pose_encoder --use_pose 1


In [ ]:
import numpy as np
Pose_Predicted='./PreTrained/poses_2020-09-18-14-03-36.npy'
Poses_3D = np.load(Pose_Predicted) 
Poses_3D = Poses_3D.astype(np.float64) 

Pose_GroundTruth='./splits/2020-09-18-14-03-36/2020-09-18-14-03-36.txt'
Pose_file_E = open(Pose_GroundTruth, 'r')

#Poses_3D_GLB = Poses_3D.copy() 
No_ref_fram = 1 
FrameIndex = 1 #  


Poses_3D_GLB = np.zeros((int(np.shape(Poses_3D)[0]/No_ref_fram), 4, 4), np.float64) 
Poses_3D_GLB[0,:,:] = np.eye(4) 
Quat_GLB = np.zeros((int(np.shape(Poses_3D)[0]/No_ref_fram), 1, 4), np.float64) 
Euler_GLB = np.zeros((int(np.shape(Poses_3D)[0]/No_ref_fram), 1, 3), np.float64) 
ii = 1 
for i in range( FrameIndex , len(Poses_3D)-No_ref_fram, No_ref_fram): 
    Poses_3D_GLB[ii,:,:] =  np.dot(Poses_3D[i-1,:,:],  Poses_3D_GLB[ii-1,:,:]) 
    ii += 1 
    #print (i) 
#plt.plot(np.squeeze(Poses_3D_GLB[1:, 0:3, 3])) 
trans_weight = 1/np.array([1., 1., 1.])


Pose_Lines_E = Pose_file_E.readlines()
#Pose_E = np.zeros((len(Pose_Lines_E), 6), np.float64)
Pose_E_Orig = np.zeros((len(Pose_Lines_E), 6), np.float64)
Start = 3
for Index_Pose in range(len(Pose_Lines_E)):  
    Pose_E_temp = Pose_Lines_E[Index_Pose].split(' ')
    Pose_E_Orig[Index_Pose][0] = float(Pose_E_temp[Start]) #- Pose_quatr[0][0]
    Pose_E_Orig[Index_Pose][1] = float(Pose_E_temp[Start+1]) #- Pose_quatr[0][1]
    Pose_E_Orig[Index_Pose][2] = float(Pose_E_temp[Start+2]) #- Pose_quatr[0][2]
    Pose_E_Orig[Index_Pose][3] = float(Pose_E_temp[Start+3]) #- Pose_quatr[0][3]
    Pose_E_Orig[Index_Pose][4] = float(Pose_E_temp[Start+4]) #- Pose_quatr[0][4]
    Pose_E_Orig[Index_Pose][5] = float(Pose_E_temp[Start+5]) #- Pose_quatr[0][5]            

plt.plot(Pose_E_Orig[:,:3] - np.mean(Pose_E_Orig[0:2,:3],0))
plt.title('GroundTruth Translation')
plt.figure()
plt.plot(np.squeeze(Poses_3D_GLB[1:, 0:3, 3]))
plt.title('Predicted Translation')
plt.show()